# Gráfica de Estandarización, Normalización y Transformación de Datos
Autora: Gladys Choque Ulloa

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer
from matplotlib.animation import FuncAnimation, PillowWriter
from scipy.stats import gaussian_kde

# --------------------------
# 1. Crear datos de ejemplo
# --------------------------
np.random.seed(42)
data_original = np.random.exponential(scale=2.0, size=100)

scaler_std = StandardScaler()
data_estandarizada = scaler_std.fit_transform(data_original.reshape(-1,1)).flatten()

scaler_minmax = MinMaxScaler()
data_normalizada = scaler_minmax.fit_transform(data_original.reshape(-1,1)).flatten()

pt = PowerTransformer(method='yeo-johnson')
data_transformada = pt.fit_transform(data_original.reshape(-1,1)).flatten()

# --------------------------
# 2. Configurar figura
# --------------------------
sns.set_style("whitegrid")
fig, axes = plt.subplots(3, 2, figsize=(14, 12))
fig.suptitle("Comparación: Original vs Transformado", fontsize=18, weight='bold')

titles = [
    "No Estandarizado ", "Estandarizado",
    "No Normalizado", "Normalizado",
    "No Transformado", "Transformado (Yeo-Johnson)"
]

data_list = [
    data_original, data_estandarizada,
    data_original, data_normalizada,
    data_original, data_transformada
]

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
n_frames = 40

# --------------------------
# 3. Función de animación
# --------------------------
def init():
    for i, ax in enumerate(axes.flatten()):
        ax.clear()
        counts, bins, _ = ax.hist(data_list[i], bins=20, color=colors[i], edgecolor='black', alpha=0.7)
        kde = gaussian_kde(data_list[i])
        x_vals = np.linspace(min(data_list[i]), max(data_list[i]), 200)
        ax.plot(x_vals, kde(x_vals)*len(data_list[i])*(bins[1]-bins[0]), color='black', lw=1.5)
        ax.set_title(titles[i], fontsize=14, weight='bold')
        ax.set_xlim(min(data_original)*0.9, max(data_original)*1.1)
        ax.set_ylim(0, 30)
    return axes.flatten()

def animate(frame):
    factor = (frame + 1)/n_frames
    for i, ax in enumerate(axes.flatten()):
        ax.clear()
        # Histograma escalado
        counts, bins = np.histogram(data_list[i], bins=20)
        scaled_counts = counts * factor
        bin_centers = 0.5*(bins[1:] + bins[:-1])
        ax.bar(bin_centers, scaled_counts, width=(bins[1]-bins[0])*0.9,
               color=colors[i], edgecolor='black', alpha=0.8)
        # Curva de densidad escalada
        kde = gaussian_kde(data_list[i])
        x_vals = np.linspace(min(data_list[i]), max(data_list[i]), 200)
        kde_scaled = kde(x_vals) * len(data_list[i]) * (bins[1]-bins[0]) * factor
        ax.plot(x_vals, kde_scaled, color='black', lw=1.5)
        ax.set_title(titles[i], fontsize=14, weight='bold')
        ax.set_xlim(min(data_original)*0.9, max(data_original)*1.1)
        ax.set_ylim(0, 30)
    return axes.flatten()

# --------------------------
# 4. Crear GIF
# --------------------------
anim = FuncAnimation(fig, animate, frames=n_frames, init_func=init, blit=False, interval=150)
gif_filename = "comparacion_datos_barras_densidad.gif"
writer = PillowWriter(fps=8)
anim.save(gif_filename, writer=writer)

plt.close(fig)
print(f"GIF guardado como {gif_filename}")
